In [58]:
import numpy as np
import scipy
import pandas as pd
import math
import random
import sklearn
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from scipy.sparse.linalg import svds
import matplotlib.pyplot as plt
import warnings 
warnings.simplefilter("ignore")

In [59]:
UserID =-9220000000000000000

In [60]:
articles_df=(pd.read_csv('C:/Users/Lee Dong woo/Documents/archive/teaable2500.csv'))

In [61]:
articles_df

,차품목,효능번호,카페인,향기준,contentId,"맛,향",personId
0,Bengal Spice,1,X,2,100001,"계피, 생강",1
1,BerryBlossom White,1,O,1,100002,과일향(베리향),2
2,Black Cherry Berry,3,X,1,100003,"과일향(체리, 베리)",3
3,Breakfast in Paris Black Tea,1,O,"1,4",100004,"라벤더꽃향, 시트러스향",4
4,ButterScotch Blondie,3,O,4,100005,"버터카라멜향, 바닐라향",5
...,...,...,...,...,...,...,...
2495,aa2496,4,O,3,102496,190aa19,2496
2496,aa2497,4,O,4,102497,401aa71,2497
2497,aa2498,3,O,1,102498,262aa31,2498
2498,aa2499,2,X,4,102499,27aa27,2499


In [62]:
articles_df['맛,향']=articles_df['맛,향'].replace('[향,' ',(,),]?' , '')
articles_df['contentId']=articles_df['contentId'].replace('a?','')
articles_df['카페인']=articles_df['카페인'].fillna('X')

In [63]:
interaction_df = pd.read_csv('C:/Users/Lee Dong woo/Documents/archive/userinteract5.csv')
interaction_df['contentId']=interaction_df['contentId'].replace('a?','')
interaction_df['contentId'].astype(float)


0        102336.0
1        102124.0
2        101361.0
3        100580.0
4        102394.0
           ...   
69863    100367.0
69864    101694.0
69865    100216.0
69866    100169.0
69867    100149.0
Name: contentId, Length: 69868, dtype: float64

In [64]:
interaction_df

,Unnamed: 0,timestamp,eventType,contentId,personId,sessionId,userAgent,userRegion,userCountry,eventStrength,interactCnt,gender,age(15~60)
0,866,1463138398,VIEW,102336,-9.223120e+18,-4.480000e+18,Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebKi...,SP,BR,1.0,43,male,18
1,1475,1463656314,VIEW,102124,-9.223120e+18,-7.820000e+18,Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebKi...,SP,BR,1.0,43,male,18
2,2916,1464113091,VIEW,101361,-9.223120e+18,-2.770000e+18,Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebKi...,SP,BR,1.0,43,male,18
3,5309,1462283851,VIEW,100580,-9.223120e+18,2.630000e+18,Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebKi...,SP,BR,1.0,43,male,18
4,9221,1462452127,VIEW,102394,-9.223120e+18,-3.670000e+18,Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebKi...,SP,BR,1.0,43,male,18
...,...,...,...,...,...,...,...,...,...,...,...,...,...
69863,71890,1482932930,VIEW,100367,9.210530e+18,-7.670000e+18,Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/53...,SP,BR,1.0,63,male,57
69864,71923,1484126240,VIEW,101694,9.210530e+18,1.220000e+18,Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/53...,SP,BR,1.0,63,male,57
69865,72100,1483015651,VIEW,100216,9.210530e+18,-2.170000e+18,Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/53...,SP,BR,1.0,63,male,57
69866,72105,1483015617,VIEW,100169,9.210530e+18,-2.170000e+18,Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/53...,SP,BR,1.0,63,male,57


In [65]:
interaction_df=interaction_df.drop('Unnamed: 0',axis=1)


In [121]:
interaction_df

,timestamp,eventType,contentId,personId,sessionId,userAgent,userRegion,userCountry,eventStrength,interactCnt,gender,age(15~60)
0,1463138398,VIEW,102336,-9.223120e+18,-4.480000e+18,Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebKi...,SP,BR,1.0,43,male,18
1,1463656314,VIEW,102124,-9.223120e+18,-7.820000e+18,Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebKi...,SP,BR,1.0,43,male,18
2,1464113091,VIEW,101361,-9.223120e+18,-2.770000e+18,Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebKi...,SP,BR,1.0,43,male,18
3,1462283851,VIEW,100580,-9.223120e+18,2.630000e+18,Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebKi...,SP,BR,1.0,43,male,18
4,1462452127,VIEW,102394,-9.223120e+18,-3.670000e+18,Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebKi...,SP,BR,1.0,43,male,18
...,...,...,...,...,...,...,...,...,...,...,...,...
69863,1482932930,VIEW,100367,9.210530e+18,-7.670000e+18,Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/53...,SP,BR,1.0,63,male,57
69864,1484126240,VIEW,101694,9.210530e+18,1.220000e+18,Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/53...,SP,BR,1.0,63,male,57
69865,1483015651,VIEW,100216,9.210530e+18,-2.170000e+18,Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/53...,SP,BR,1.0,63,male,57
69866,1483015617,VIEW,100169,9.210530e+18,-2.170000e+18,Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/53...,SP,BR,1.0,63,male,57


In [126]:
k = interaction_df[interaction_df['personId']==-9.223120e+18	]
k

,timestamp,eventType,contentId,personId,sessionId,userAgent,userRegion,userCountry,eventStrength,interactCnt,gender,age(15~60)
0,1463138398,VIEW,102336,-9.223120e+18,-4.480000e+18,Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebKi...,SP,BR,1.0,43,male,18
1,1463656314,VIEW,102124,-9.223120e+18,-7.820000e+18,Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebKi...,SP,BR,1.0,43,male,18
2,1464113091,VIEW,101361,-9.223120e+18,-2.770000e+18,Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebKi...,SP,BR,1.0,43,male,18
3,1462283851,VIEW,100580,-9.223120e+18,2.630000e+18,Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebKi...,SP,BR,1.0,43,male,18
4,1462452127,VIEW,102394,-9.223120e+18,-3.670000e+18,Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebKi...,SP,BR,1.0,43,male,18
...,...,...,...,...,...,...,...,...,...,...,...,...
58,1487677784,VIEW,100593,-9.223120e+18,3.230000e+18,Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/53...,SP,BR,1.0,43,male,18
59,1487678254,VIEW,100332,-9.223120e+18,3.230000e+18,Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/53...,SP,BR,1.0,43,male,18
60,1487677800,VIEW,102059,-9.223120e+18,3.230000e+18,Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/53...,SP,BR,1.0,43,male,18
61,1487347947,VIEW,100050,-9.223120e+18,-4.560000e+16,Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/53...,SP,BR,1.0,43,male,18


In [68]:
event_type_strength = {
  'VIEW': 1.0,
   'LIKE': 2.0, 
   'BOOKMARK': 2.5, 
   'FOLLOW': 3.0,
   'COMMENT CREATED': 4.0,  
}

interaction_df['eventStrength'] = (
  interaction_df
    .loc[:, 'eventType']
    .apply(lambda d: event_type_strength[d])
)


In [69]:
interaction_df_over5 = (interaction_df
  .groupby('personId', group_keys=False)
  .apply(lambda df: df.assign(interactCnt = lambda d: d['contentId'].nunique()))
  .loc[lambda d: d['interactCnt'] >= 5]
)
interaction_df_over5

,timestamp,eventType,contentId,personId,sessionId,userAgent,userRegion,userCountry,eventStrength,interactCnt,gender,age(15~60)
0,1463138398,VIEW,102336,-9.223120e+18,-4.480000e+18,Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebKi...,SP,BR,1.0,62,male,18
1,1463656314,VIEW,102124,-9.223120e+18,-7.820000e+18,Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebKi...,SP,BR,1.0,62,male,18
2,1464113091,VIEW,101361,-9.223120e+18,-2.770000e+18,Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebKi...,SP,BR,1.0,62,male,18
3,1462283851,VIEW,100580,-9.223120e+18,2.630000e+18,Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebKi...,SP,BR,1.0,62,male,18
4,1462452127,VIEW,102394,-9.223120e+18,-3.670000e+18,Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebKi...,SP,BR,1.0,62,male,18
...,...,...,...,...,...,...,...,...,...,...,...,...
69863,1482932930,VIEW,100367,9.210530e+18,-7.670000e+18,Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/53...,SP,BR,1.0,128,male,57
69864,1484126240,VIEW,101694,9.210530e+18,1.220000e+18,Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/53...,SP,BR,1.0,128,male,57
69865,1483015651,VIEW,100216,9.210530e+18,-2.170000e+18,Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/53...,SP,BR,1.0,128,male,57
69866,1483015617,VIEW,100169,9.210530e+18,-2.170000e+18,Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/53...,SP,BR,1.0,128,male,57


In [70]:
interaction_full_df = (
  interaction_df_over5
    .groupby(['personId', 'contentId'], as_index=False)['eventStrength']
    .sum()
    .assign(eventScore = lambda d: np.log2(1+d['eventStrength']))
)

interaction_full_df.head(10)

,personId,contentId,eventStrength,eventScore
0,-9.223120e+18,100050,1.0,1.0
1,-9.223120e+18,100205,1.0,1.0
2,-9.223120e+18,100214,1.0,1.0
3,-9.223120e+18,100332,1.0,1.0
4,-9.223120e+18,100380,1.0,1.0
5,-9.223120e+18,100416,1.0,1.0
6,-9.223120e+18,100427,1.0,1.0
7,-9.223120e+18,100433,1.0,1.0
8,-9.223120e+18,100487,1.0,1.0
9,-9.223120e+18,100498,1.0,1.0


# 평가

In [71]:
interaction_train, interaction_test = train_test_split(
    interaction_full_df,
    stratify=interaction_full_df['personId'],
    test_size=0.2,
    random_state=40
)

In [72]:
interaction_full_indexed = interaction_full_df.set_index('personId')
interaction_train_indexed = interaction_train.set_index('personId')
interaction_test_indexed = interaction_test.set_index('personId')
interaction_df

,timestamp,eventType,contentId,personId,sessionId,userAgent,userRegion,userCountry,eventStrength,interactCnt,gender,age(15~60)
0,1463138398,VIEW,102336,-9.223120e+18,-4.480000e+18,Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebKi...,SP,BR,1.0,43,male,18
1,1463656314,VIEW,102124,-9.223120e+18,-7.820000e+18,Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebKi...,SP,BR,1.0,43,male,18
2,1464113091,VIEW,101361,-9.223120e+18,-2.770000e+18,Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebKi...,SP,BR,1.0,43,male,18
3,1462283851,VIEW,100580,-9.223120e+18,2.630000e+18,Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebKi...,SP,BR,1.0,43,male,18
4,1462452127,VIEW,102394,-9.223120e+18,-3.670000e+18,Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebKi...,SP,BR,1.0,43,male,18
...,...,...,...,...,...,...,...,...,...,...,...,...
69863,1482932930,VIEW,100367,9.210530e+18,-7.670000e+18,Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/53...,SP,BR,1.0,63,male,57
69864,1484126240,VIEW,101694,9.210530e+18,1.220000e+18,Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/53...,SP,BR,1.0,63,male,57
69865,1483015651,VIEW,100216,9.210530e+18,-2.170000e+18,Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/53...,SP,BR,1.0,63,male,57
69866,1483015617,VIEW,100169,9.210530e+18,-2.170000e+18,Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/53...,SP,BR,1.0,63,male,57


In [73]:
interaction_full_indexed

,contentId,eventStrength,eventScore
personId,,,
-9.223120e+18,100050,1.0,1.000000
-9.223120e+18,100205,1.0,1.000000
-9.223120e+18,100214,1.0,1.000000
-9.223120e+18,100332,1.0,1.000000
-9.223120e+18,100380,1.0,1.000000
...,...,...,...
9.210530e+18,102280,4.0,2.321928
9.210530e+18,102351,1.0,1.000000
9.210530e+18,102422,1.0,1.000000


In [74]:
def get_items_interacted(person_id, interaction_df):
    interacted_items = interaction_df.loc[person_id]['contentId']
    return set(interacted_items if type(interacted_items) == pd.Series else [interacted_items])
#get_items_interacted로 person_id와 입력받은 person_id의 contentId를 Series로 받아옴

In [75]:
interaction_df

,timestamp,eventType,contentId,personId,sessionId,userAgent,userRegion,userCountry,eventStrength,interactCnt,gender,age(15~60)
0,1463138398,VIEW,102336,-9.223120e+18,-4.480000e+18,Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebKi...,SP,BR,1.0,43,male,18
1,1463656314,VIEW,102124,-9.223120e+18,-7.820000e+18,Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebKi...,SP,BR,1.0,43,male,18
2,1464113091,VIEW,101361,-9.223120e+18,-2.770000e+18,Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebKi...,SP,BR,1.0,43,male,18
3,1462283851,VIEW,100580,-9.223120e+18,2.630000e+18,Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebKi...,SP,BR,1.0,43,male,18
4,1462452127,VIEW,102394,-9.223120e+18,-3.670000e+18,Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebKi...,SP,BR,1.0,43,male,18
...,...,...,...,...,...,...,...,...,...,...,...,...
69863,1482932930,VIEW,100367,9.210530e+18,-7.670000e+18,Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/53...,SP,BR,1.0,63,male,57
69864,1484126240,VIEW,101694,9.210530e+18,1.220000e+18,Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/53...,SP,BR,1.0,63,male,57
69865,1483015651,VIEW,100216,9.210530e+18,-2.170000e+18,Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/53...,SP,BR,1.0,63,male,57
69866,1483015617,VIEW,100169,9.210530e+18,-2.170000e+18,Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/53...,SP,BR,1.0,63,male,57


In [79]:
for idx, person_id in enumerate(list(interaction_test_indexed.index.unique().values)):
    print (person_id)
  

-1.57829e+18
2.41628e+18
-1.03202e+18
8.69821e+18
2.75457e+18
-2.97988e+18
7.6964e+18
-7.09288e+17
-5.92776e+18
-5.3455e+17
2.71782e+17
-2.92718e+17
-4.29818e+18
2.95625e+18
3.94383e+18
-8.67075e+18
-6.17407e+18
-6.15301e+18
3.94872e+18
-7.45649e+18
2.14655e+18
3.95118e+18
-1.08842e+17
-5.10621e+17
-8.3367e+18
4.00323e+18
8.61525e+18
-3.47415e+18
-3.37465e+18
6.12011e+18
-5.54845e+18
1.81178e+18
6.52551e+18
1.12007e+18
7.04749e+18
3.60919e+18
-8.71946e+18
-9.15634e+18
-8.55017e+18
6.46547e+18
-1.60283e+18
-8.8453e+18
-2.39979e+18
7.39299e+18
2.00208e+18
3.21701e+18
8.23733e+18
3.09451e+18
-1.29279e+18
-8.43024e+18
-5.52715e+18
-6.06712e+18
-3.4529e+18
7.94808e+18
4.43237e+18
7.59109e+18
3.37538e+18
-7.36552e+18
4.25415e+18
-3.53527e+18
3.63691e+18
-9.01653e+18
1.36506e+18
5.12707e+18
-1.6169e+18
-2.97954e+18
-8.62076e+18
8.50836e+18
-5.86738e+18
1.11612e+18
4.87019e+18
7.29303e+18
-3.39005e+18
-5.9186e+18
-4.16017e+18
-1.44364e+18
4.31305e+18
6.64412e+18
8.11332e+18
-5.37596e+18
9.2105

In [81]:
interaction_test_indexed

,contentId,eventStrength,eventScore
personId,,,
-1.578290e+18,102476,1.0,1.000000
2.416280e+18,100219,1.0,1.000000
-1.032020e+18,101398,1.0,1.000000
8.698210e+18,100025,1.0,1.000000
2.754570e+18,102444,1.0,1.000000
...,...,...,...
-1.443640e+18,101213,1.0,1.000000
8.759620e+18,101437,2.0,1.584963
8.872820e+18,101034,1.0,1.000000


In [82]:
class ModelEvaluator:
    def __init__(self, n_non_interacted=3): #여기서 우리는 티가 70개 밖에 되지 않음. 따라서 상호작용하지 않은 애들을 15개로 잡을 예정
        self.EVAL_RANDOM_SAMPLE_NON_INTERACTED_ITEMS = n_non_interacted 
        #ModelEvaluator 실행시 가장 먼저 실행되는 부분
        #해당 인스턴스의 random_sample_non_interacted_items 개수 설정.
        
    def get_non_interacted_items_sample(self, person_id, sample_size, seed=40):
        interacted_items = get_items_interacted(person_id, interaction_full_indexed) #인덱싱 된 interaction_full_indexed를 가지고 
        #get_items_interacted 실행
        all_items = set(articles_df['contentId']) # 티의 content_id를 중복을 제외한 상태인 set집합으로 가져옴
        non_interacted_items = set(all_items - interacted_items) #상호작용이 없었던 애들을 전체 집합에서 빼서 가져옴
        
        random.seed(seed) #랜덤시드 초기화
        non_interacted_items_sample = random.sample(non_interacted_items, sample_size) #sample_size만큼의 non_interacted_items를 랜덤추출
       
        return set(non_interacted_items_sample) #중복을 제외한 애들을 샘플집합으로 가져옴
        
    def _verify_hit_top_n(self, item_id, recommend_items, topn):
        try:
            index = next(i for i, c in enumerate(recommend_items) if c == item_id)
        except:
            index = -1 
        hit = int(index in range(0, topn))
        return hit, index
    
    
    def evaluate_model_for_user(self, model, person_id):
        interacted_values_testset = interaction_test_indexed.loc[person_id]
        if type(interacted_values_testset['contentId']) == pd.Series:
            person_interacted_items_testset = set(interacted_values_testset['contentId'])
        else:
            person_interacted_items_testset = set([int(interacted_values_testset['contentId'])])
        #유저별 event한번이라도 발생한 item의 아이템셋 집합 가져오기
        interacted_items_count_testset = len(person_interacted_items_testset)
        #개수
        # 특정 사용자에 대한 추천 순위 목록을 받아온다
        person_recs = model.recommend_items(
            person_id,
            items_to_ignore=get_items_interacted(person_id, interaction_train_indexed),
            topn=10
        )
        
        hits_at_5_count = 0
        hits_at_10_count = 0
        
        # test셋에서 사용자가 상호작용한 모든 항목에 대해 반복한다
        for item_id in person_interacted_items_testset:
            
            # 사용자가 상호작용하지 않은 n개 항목을 샘플링한다
            non_interacted_items_sample = self.get_non_interacted_items_sample(
                person_id,
                sample_size=self.EVAL_RANDOM_SAMPLE_NON_INTERACTED_ITEMS,
                seed=item_id %(2**32)
            )
            
            # 현재 선택한 item_id(상호작용 있었던 항목)와 15개 랜덤 샘플을 합친다
            items_to_filter_recs = non_interacted_items_sample.union(set([item_id]))
            
            # 추천 결과물 중에서 현재 선택한 item_id와 15개 랜덤 샘플의 결과물로만 필터링한다
            valid_recs_df = person_recs[person_recs['contentId'].isin(items_to_filter_recs)]# 상호작용 있었던 항목과 15개 랜덤샘플을 합친
            #것을 탐색해 해당 목록에 있는 컨텐츠만 가져옴// 유효한 추천목록을 가져옴
            # 추천목록은 일단 
            valid_recs = valid_recs_df['contentId'].values
            
            # 현재 선택한 item_id가 Top-N 추천 결과 안에 있는지 확인한다
            hit_at_5, index_at_5 = self._verify_hit_top_n(item_id, valid_recs, 5)
            hits_at_5_count += hit_at_5
            hit_at_10, index_at_10 = self._verify_hit_top_n(item_id, valid_recs, 10)
            hits_at_10_count += hit_at_10
            
        # Recall 값은 상호작용 있었던 항목들 중에서 관련없는 항목들과 섞였을 때 Top-N에 오른 항목들의 비율로 나타낼 수 있다
        recall_at_5 = hits_at_5_count / interacted_items_count_testset
        recall_at_10 = hits_at_10_count / interacted_items_count_testset
        #여기서는 직접적으로 평가지표를 사용할수가 없음. 왜냐하면 0,1 등으로 일치 여부를 따질수가 없음.
        #이걸 하려면 원-핫 인코딩을 해야하는데, 그거하기 귀찮아서 그렇게 한듯?
        #이부분은 recall@, precision@ 등의 방식으로 해결이 가능함
        #rank-aware 지표를 확인하면 좋을듯
        #그래서 임의적으로 recall지표를 식으로서 표현한 것.
        person_metrics = {
            'hits@5_count': hits_at_5_count,
            'hits@10_count': hits_at_10_count,
            'interacted_count': interacted_items_count_testset,
            'recall@5': recall_at_5,
            'recall@10': recall_at_10
        }
        return person_metrics
    
    def evaluate_model(self, model):
        
        people_metrics = []
        for idx, person_id in enumerate(list(interaction_test_indexed.index.unique().values)):
            person_metrics = self.evaluate_model_for_user(model, person_id)#변수통일
            person_metrics['_person_id'] = person_id
            people_metrics.append(person_metrics)# 각 personId별로 추출해서 합치기

        print('{} users processed'.format(idx))
        
        detailed_result = (
            pd.DataFrame(people_metrics)
              .sort_values('interacted_count', ascending=False) #카운트 개수순으로 정렬, 내림차순
        )
        
        global_recall_at_5 = detailed_result['hits@5_count'].sum() / detailed_result['interacted_count'].sum()
        global_recall_at_10 = detailed_result['hits@10_count'].sum() / detailed_result['interacted_count'].sum()
        
        global_metrics = {
            'model_name': model.get_model_name(),
            'recall@5': global_recall_at_5,
            'recall@10': global_recall_at_10
        }
        
        return global_metrics, detailed_result

In [83]:
import nltk
vectorizer = TfidfVectorizer(
    analyzer='word',
    ngram_range=(1, 2),
    min_df=0.003,
    max_df=0.5,
    max_features=1000
    
)

In [84]:
item_ids = articles_df['contentId'].tolist()
tfidf_matrix = vectorizer.fit_transform(articles_df['맛,향'])
tfidf_feature_names = vectorizer.get_feature_names()

In [85]:
tfidf_matrix

<2500x3 sparse matrix of type '<class 'numpy.float64'>'
	with 36 stored elements in Compressed Sparse Row format>

In [86]:
def get_item_profile(item_id):
    idx = item_ids.index(item_id)
    item_profile = tfidf_matrix[idx:idx+1]
    return item_profile

def get_item_profiles(ids):
    item_profiles_list = [get_item_profile(x) for x in ids]
    item_profiles = scipy.sparse.vstack(item_profiles_list)
    return item_profiles

def build_user_profile(person_id, interaction_indexed_df):
    interaction_person_df = interaction_indexed_df.loc[person_id]
    user_item_profiles = get_item_profiles(interaction_person_df['contentId'])
    
    user_item_strengths = np.array(interaction_person_df['eventStrength']).reshape(-1, 1)
    
    # 상호작용 강도를 바탕으로 가중치를 부여하여 평균 계산한다
    user_item_strengths_weighted_avg = \
        np.sum(user_item_profiles.multiply(user_item_strengths), axis=0) /\
        np.sum(user_item_strengths)
        
    user_profile_norm = sklearn.preprocessing.normalize(user_item_strengths_weighted_avg)
    return user_profile_norm

def build_user_profiles():
    interaction_indexed_df = (interaction_full_df
        .loc[lambda d: d['contentId'].isin(articles_df['contentId'])]
        .set_index('personId')
    )
    user_profiles = {}
    
    for person_id in interaction_indexed_df.index.unique():
        user_profiles[person_id] = build_user_profile(person_id, interaction_indexed_df)
        
    return user_profiles

In [87]:
user_profiles = build_user_profiles()
len(user_profiles)

1140

In [88]:
myprofile = user_profiles[-9223120000000000000].flatten().tolist()
A= pd.DataFrame(sorted(zip(tfidf_feature_names, myprofile), key=lambda x: -x[1])[:20],
             columns=['token', 'relevance'])
A

,token,relevance
0,계피,0.0
1,과일향,0.0
2,생강,0.0


# 인기도 모형

In [89]:
item_popularity = (interaction_full_df
 .groupby('contentId')['eventStrength'].sum()
 .sort_values(ascending=False)
 .reset_index()
)

item_popularity.head(10)

,contentId,eventStrength
0,101277,61.0
1,100215,60.0
2,102020,60.0
3,100731,59.5
4,101896,58.0
5,102115,57.5
6,100567,56.5
7,101079,56.0
8,101591,56.0
9,102257,55.5


In [90]:
class PopularityRecommender:
    
    MODEL_NAME = 'Popularity'
    
    def __init__(self, popularity_df, items_df=None):
        self.popularity_df = popularity_df
        self.items_df = items_df
        
    def get_model_name(self):
        return self.MODEL_NAME
    
    def recommend_items(self, user_id, items_to_ignore=[], topn=10, verbose=False):
        # 인기상품 중에서 사용자가 보지 않았던 상품을 추천한다
        recommendations = (
          self.popularity_df[~self.popularity_df['contentId'].isin(items_to_ignore)]
            .sort_values('eventStrength', ascending=False)
            .head(topn)
        )
        
        if verbose:
            if self.items_df is None:
                raise Exception('"items_df" is required in verbose mode')
            recommendations = (recommendations
                .merge(self.items_df, how='left', left_on='contentId', right_on='contentId')
                .loc[:, ['eventStrength', 'contentId', 'title', 'url', 'lang']]
            )
            
        return recommendations

# ContentBasedRecommender

In [91]:
class ContentBasedRecommender:
    
    MODEL_NAME = 'Content-Based'
    
    def __init__(self, item_ids, items_df=None):
        self.item_ids = item_ids
        self.items_df = items_df
        
    def get_model_name(self):
        return self.MODEL_NAME
    
    def _get_similar_items_to_user_profile(self, person_id, topn=1000):
        # 유저 특성과 항목 특성 사이의 코사인 유사도를 구한다
        cosine_similarities = cosine_similarity(user_profiles[person_id], tfidf_matrix)
        
        # 가장 유사한 항목을 찾는다
        similar_indices = cosine_similarities.argsort().flatten()[-topn:]
        
        # 유사도를 기준으로 유사한 항목을 정렬한다
        similar_items = sorted(
            [(item_ids[i], cosine_similarities[0, i]) for i in similar_indices],
            key=lambda x: -x[1]
        )
        
        return similar_items
    
    def recommend_items(self, user_id, items_to_ignore=[], topn=10, verbose=False):
        similar_items = self._get_similar_items_to_user_profile(user_id)
        
        # 기존에 상호작용했던 항목은 제거한다
        similar_items_filtered = list(filter(lambda x: x[0] not in items_to_ignore, similar_items))
        
        recommendations = (
            pd.DataFrame(similar_items_filtered, columns=['contentId', 'recStrength'])
              .head(topn)
        )
        
        if verbose:
            if self.items_df is None:
                 raise Exception('"items_df" is required in verbose mode')
            recommendations = (recommendations
                .merge(self.items_df, how='left', left_on='contentId', right_on='contentId')
                .loc[:, ['recStrength', 'contentId', 'title', 'url', 'lang']]
            )
        
        return recommendations

In [92]:
popularity_model = PopularityRecommender(item_popularity, articles_df)

In [93]:
interaction_train_indexed

,contentId,eventStrength,eventScore
personId,,,
-3.596630e+18,101257,2.0,1.584963
4.537320e+18,100709,2.0,1.584963
7.308880e+18,102002,1.0,1.000000
1.116120e+18,100639,1.0,1.000000
-2.626630e+18,100901,1.0,1.000000
...,...,...,...
-8.763400e+18,102407,1.0,1.000000
-8.020830e+18,102393,1.0,1.000000
-4.996400e+18,101219,1.0,1.000000


In [94]:
content_based_model = ContentBasedRecommender(item_ids, articles_df)

In [95]:
POPULARITY=popularity_model.recommend_items( 2, topn=70, verbose=False)
POPULARITY['contentId']=POPULARITY['contentId'].astype(float)
POPULARITY['eventStrength']=POPULARITY['eventStrength'].astype(float)
POPULARITY

,contentId,eventStrength
0,101277.0,61.0
2,102020.0,60.0
1,100215.0,60.0
3,100731.0,59.5
4,101896.0,58.0
...,...,...
65,100219.0,49.0
66,101150.0,49.0
67,100491.0,49.0
68,101659.0,49.0


In [96]:
item_popularity

,contentId,eventStrength
0,101277,61.0
1,100215,60.0
2,102020,60.0
3,100731,59.5
4,101896,58.0
...,...,...
2495,100590,15.5
2496,102222,15.0
2497,100348,14.5
2498,101525,14.5


In [97]:
item_popularity['contentId']=item_popularity['contentId'].astype(float)
item_popularity.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2500 entries, 0 to 2499
Data columns (total 2 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   contentId      2500 non-null   float64
 1   eventStrength  2500 non-null   float64
dtypes: float64(2)
memory usage: 39.2 KB


In [98]:
POPULARITY.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 70 entries, 0 to 69
Data columns (total 2 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   contentId      70 non-null     float64
 1   eventStrength  70 non-null     float64
dtypes: float64(2)
memory usage: 1.6 KB


In [99]:
users_items_pivot_df = (interaction_train
  .pivot(index='personId', columns='contentId', values='eventStrength')
  .fillna(0)
)

In [100]:
users_items_pivot_matrix = users_items_pivot_df.values
users_items_pivot_matrix[:10]

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [101]:
NUMBER_OF_FACTORS_MF = 5
U, sigma, Vt = svds(users_items_pivot_matrix, k=NUMBER_OF_FACTORS_MF)
sigma_mat = np.diag(sigma)
all_user_predicted_ratings = np.dot(np.dot(U, sigma_mat), Vt)
all_user_predicted_ratings

array([[ 0.04565279,  0.0204874 ,  0.01035344, ...,  0.0241808 ,
         0.00059986,  0.01677388],
       [ 0.00583258,  0.00283095,  0.00108374, ...,  0.00140629,
        -0.00106924,  0.0029024 ],
       [ 0.03006068,  0.00963305,  0.00529887, ...,  0.00755084,
         0.00381638,  0.0037779 ],
       ...,
       [ 0.00530462,  0.00426559,  0.01526332, ...,  0.0065335 ,
         0.01177753,  0.00434335],
       [ 0.01162599,  0.00590024,  0.00466436, ...,  0.0114229 ,
         0.00174537,  0.00522908],
       [ 0.09414482,  0.0404261 ,  0.02777717, ...,  0.0595022 ,
         0.00993645,  0.03087362]])

In [102]:
user_ids = list(users_items_pivot_df.index)
user_ids[:10]

cf_preds_df = (
  pd.DataFrame(all_user_predicted_ratings, 
               columns=users_items_pivot_df.columns, 
               index=user_ids)
    .transpose()
)

# Collaborative Filtering

In [103]:
class CFRecommender:
    
    MODEL_NAME = 'Collaborative Filtering'
    
    def __init__(self, cf_predictions_df, items_df=None):
        self.cf_predictions_df = cf_predictions_df
        self.items_df = items_df
        
    def get_model_name(self):
        return self.MODEL_NAME
    
    def recommend_items(self, user_id, items_to_ignore=[], topn=10, verbose=False):
        # 사용자에 대한 예측값을 가져와서 정렬한다
        sorted_user_prediction = (self.cf_predictions_df
            .loc[:, user_id]
            .sort_values(ascending=False)
            .reset_index()
            .rename(columns={user_id: 'recStrength'})
        )
        
        recommendations = (sorted_user_prediction
            .loc[lambda d: ~d['contentId'].isin(items_to_ignore)]
            .sort_values('recStrength', ascending=False)
            .head(topn)
        )
        
        if verbose:
            if self.item_df is None:
                raise Exception('"items_df" is required in verbose mode')
            
            recommendations = (recommendations
                .merge(self.items_df, how='left', left_on='contentId', right_on='contentId')
                .loc[:, ['recStrength', 'contentId', 'title', 'url', 'lang']]
            )
            
        return recommendations

In [104]:
cf_recommender_model = CFRecommender(cf_preds_df, articles_df)

# Hybrid

In [105]:
class HybridRecommender:
    
    MODEL_NAME = 'Hybrid'
    
    def __init__(self, cb_rec_model, cf_rec_model, items_df):
        self.cb_rec_model = cb_rec_model
        self.cf_rec_model = cf_rec_model
        self.items_df = items_df
        
    def get_model_name(self):
        return self.MODEL_NAME
    
    def recommend_items(self, user_id, items_to_ignore=[], topn=10, verbose=False):
        # 상위 1000개의 컨텐츠 기반 모형 추천을 가져온다
        cb_recs = (self.cb_rec_model
            .recommend_items(user_id, items_to_ignore=items_to_ignore, verbose=verbose, topn=1000)
            .rename(columns={'recStrength': 'recStrengthCB'})
        )
        
        # 상위 1000개의 협업필터링 추천을 가져온다
        cf_recs = (self.cf_rec_model
            .recommend_items(user_id, items_to_ignore=items_to_ignore, verbose=verbose, topn=1000)
            .rename(columns={'recStrength': 'recStrengthCF'})
        )
        
        # 1) 두 모형의 결과를 합친다
        # 2) CF, CB 모형의 점수를 바탕으로 Hybrid 모형의 점수를 계산한다
        # 3) Hybrid 점수를 기준으로 정렬한다
        recommendations = (cb_recs
            .merge(cf_recs, how='inner', left_on='contentId', right_on='contentId')
            .assign(recStrengthHybrid = lambda d: d['recStrengthCB'] * d['recStrengthCF'])
            .sort_values('recStrengthHybrid', ascending=False)
            .head(topn)
        )
        
        if verbose:
            if self.items_df is None:
                raise Exception('"items_df" is required in verbose mode')
            
            recommendations = (recommendations
                .merge(self.items_df, how='left', left_on='contentId', right_on='contentId')
                .loc[:, ['recStrengthHybrid', 'contentId', 'title', 'url', 'lang']]
            )
        
        return recommendations

In [106]:
hybrid_recommender_model = HybridRecommender(content_based_model, cf_recommender_model, articles_df)

In [107]:
interaction_test_indexed.info()

<class 'pandas.core.frame.DataFrame'>
Float64Index: 13144 entries, -1.57829e+18 to 3.60919e+18
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   contentId      13144 non-null  int64  
 1   eventStrength  13144 non-null  float64
 2   eventScore     13144 non-null  float64
dtypes: float64(2), int64(1)
memory usage: 668.8 KB


In [108]:
interaction_train_indexed.info()

<class 'pandas.core.frame.DataFrame'>
Float64Index: 52572 entries, -3.59663e+18 to -4.81618e+16
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   contentId      52572 non-null  int64  
 1   eventStrength  52572 non-null  float64
 2   eventScore     52572 non-null  float64
dtypes: float64(2), int64(1)
memory usage: 1.6 MB


In [109]:
interaction_test_indexed['contentId']=interaction_test_indexed['contentId'].astype(float)

In [110]:
interaction_test_indexed.info()

<class 'pandas.core.frame.DataFrame'>
Float64Index: 13144 entries, -1.57829e+18 to 3.60919e+18
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   contentId      13144 non-null  float64
 1   eventStrength  13144 non-null  float64
 2   eventScore     13144 non-null  float64
dtypes: float64(3)
memory usage: 668.8 KB


In [111]:
interaction_train_indexed['contentId']=interaction_train_indexed['contentId'].astype(float)

In [112]:
model_evaluator = ModelEvaluator(n_non_interacted=15)

In [113]:
print('Popularity 추천 모형을 평가합니다')
pop_global_metrics, pop_detailed_results = model_evaluator.evaluate_model(popularity_model)
print('Global Metrics:\n{}'.format(pop_global_metrics))
pop_detailed_results.head(10)

Popularity 추천 모형을 평가합니다
1139 users processed
Global Metrics:
{'model_name': 'Popularity', 'recall@5': 0.007988435788192331, 'recall@10': 0.007988435788192331}


,hits@5_count,hits@10_count,interacted_count,recall@5,recall@10,_person_id
2,2,2,264,0.007576,0.007576,-1.032020e+18
75,2,2,235,0.008511,0.008511,-1.443640e+18
35,2,2,221,0.009050,0.009050,3.609190e+18
116,2,2,177,0.011299,0.011299,-2.626630e+18
101,3,3,150,0.020000,0.020000,-3.596630e+18
139,2,2,143,0.013986,0.013986,3.302560e+18
7,1,1,141,0.007092,0.007092,-7.092880e+17
60,0,0,126,0.000000,0.000000,3.636910e+18
1,0,0,123,0.000000,0.000000,2.416280e+18
5,0,0,120,0.000000,0.000000,-2.979880e+18


In [116]:
print('Collaborative Filtering 추천 모형을 평가합니다')
pop_global_metrics, pop_detailed_results = model_evaluator.evaluate_model(cf_recommender_model)
print('Global Metrics:\n{}'.format(pop_global_metrics))
pop_detailed_results.head(10)

Collaborative Filtering 추천 모형을 평가합니다
1139 users processed
Global Metrics:
{'model_name': 'Collaborative Filtering', 'recall@5': 0.005097382836275107, 'recall@10': 0.005097382836275107}


,hits@5_count,hits@10_count,interacted_count,recall@5,recall@10,_person_id
2,3,3,264,0.011364,0.011364,-1.032020e+18
75,1,1,235,0.004255,0.004255,-1.443640e+18
35,1,1,221,0.004525,0.004525,3.609190e+18
116,2,2,177,0.011299,0.011299,-2.626630e+18
101,0,0,150,0.000000,0.000000,-3.596630e+18
139,0,0,143,0.000000,0.000000,3.302560e+18
7,2,2,141,0.014184,0.014184,-7.092880e+17
60,1,1,126,0.007937,0.007937,3.636910e+18
1,1,1,123,0.008130,0.008130,2.416280e+18
5,1,1,120,0.008333,0.008333,-2.979880e+18


In [115]:
print('Popularity 추천 모형을 평가합니다')
pop_global_metrics, pop_detailed_results = model_evaluator.evaluate_model(hybrid_recommender_model )
print('Global Metrics:\n{}'.format(pop_global_metrics))
pop_detailed_results.head(10)

Popularity 추천 모형을 평가합니다
1139 users processed
Global Metrics:
{'model_name': 'Hybrid', 'recall@5': 0.006238587948874011, 'recall@10': 0.006238587948874011}


,hits@5_count,hits@10_count,interacted_count,recall@5,recall@10,_person_id
2,2,2,264,0.007576,0.007576,-1.032020e+18
75,1,1,235,0.004255,0.004255,-1.443640e+18
35,1,1,221,0.004525,0.004525,3.609190e+18
116,0,0,177,0.000000,0.000000,-2.626630e+18
101,0,0,150,0.000000,0.000000,-3.596630e+18
139,1,1,143,0.006993,0.006993,3.302560e+18
7,1,1,141,0.007092,0.007092,-7.092880e+17
60,2,2,126,0.015873,0.015873,3.636910e+18
1,0,0,123,0.000000,0.000000,2.416280e+18
5,1,1,120,0.008333,0.008333,-2.979880e+18


# Recall 지표와 Precision 지표를 활용한 F1 Score

In [127]:
print('Popularity 추천 모형을 평가합니다')
pop_global_metrics, pop_detailed_results = model_evaluator.evaluate_model(content_based_model )
print('Global Metrics:\n{}'.format(pop_global_metrics))
pop_detailed_results.head(10)

Popularity 추천 모형을 평가합니다
1139 users processed
Global Metrics:
{'model_name': 'Content-Based', 'recall@5': 0.009129640900791236, 'recall@10': 0.009129640900791236}


,hits@5_count,hits@10_count,interacted_count,recall@5,recall@10,_person_id
2,2,2,264,0.007576,0.007576,-1.032020e+18
75,2,2,235,0.008511,0.008511,-1.443640e+18
35,1,1,221,0.004525,0.004525,3.609190e+18
116,0,0,177,0.000000,0.000000,-2.626630e+18
101,0,0,150,0.000000,0.000000,-3.596630e+18
139,1,1,143,0.006993,0.006993,3.302560e+18
7,1,1,141,0.007092,0.007092,-7.092880e+17
60,1,1,126,0.007937,0.007937,3.636910e+18
1,0,0,123,0.000000,0.000000,2.416280e+18
5,0,0,120,0.000000,0.000000,-2.979880e+18
